In [1]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import scipy.io
mat = scipy.io.loadmat('/home/aniruddha/Documents/data_10feature.mat')
arr = mat['TR1_10feature']
arr = np.array(arr)
arr = arr.reshape(-1)
print(arr.shape)

X_train = []
for i in range(0,14):
    for j in range(0,arr[i].shape[0]):
        X_train.append(arr[i][j])
X_train = np.array(X_train)
print(X_train.shape)

y_train = []
for i in range(0,arr.shape[0]):
    for j in range(0,arr[i].shape[0]):
        y_train.append(i)
y_train = np.array(y_train)
print(y_train.shape)
print(y_train[1])


(14,)
(1242, 10)
(1242,)
0


In [3]:
arr1 = mat['TS1_10feature']
arr1 = np.array(arr1)
arr1 = arr1.reshape(-1)
print(arr1.shape)

X_test = []
for i in range(0,14):
    for j in range(0,arr1[i].shape[0]):
        X_test.append(arr1[i][j])
X_test = np.array(X_test)
print(X_test.shape)

y_test = []
for i in range(0,arr1.shape[0]):
    for j in range(0,arr1[i].shape[0]):
        y_test.append(i)
y_test = np.array(y_test)
print(y_test.shape)
print(y_test[1])


(14,)
(1252, 10)
(1252,)
0


In [4]:
arr2 = mat['TS2_10feature']
arr2 = np.array(arr2)
arr2 = arr2.reshape(-1)
print(arr2.shape)

X_test1 = []
for i in range(0,14):
    for j in range(0,arr2[i].shape[0]):
        X_test1.append(arr2[i][j])
X_test1 = np.array(X_test1)
print(X_test1.shape)

y_test1 = []
for i in range(0,arr2.shape[0]):
    for j in range(0,arr2[i].shape[0]):
        y_test1.append(i)
y_test1 = np.array(y_test1)
print(y_test1.shape)
print(y_test1[1])


(14,)
(627, 10)
(627,)
0


In [5]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_test1 = X_test1.astype('float32')

X_train = X_train/10000
X_test = X_test/10000
X_test1 = X_test1/10000

print(X_train.max())
print(X_test.max())
print(X_test1.max())

0.6707
0.6804
0.789


In [6]:

def create_addi_pairs(x, y):
    pairs = []
    labels = []
    for i in range(0,100):
        k1 = k1 = random.randrange(0,x.shape[0])
        for j in range(0,5):
            k2 = random.randrange(0, y.shape[0])
            pairs+= [[x[k1],y[k2]]]
            labels += [3]
    return np.array(pairs), np.array(labels)

def create_pairs(x, y, digit_indices):
    
    pairs = []
    labels = []
    labels1 = []
    n = min([len(digit_indices[d]) for d in range(10)]) - 1
    for d in range(10):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            labels1 += [[y[z1], y[z2]]]
            inc = random.randrange(1, 10)
            dn = (d + inc) % 10
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels1 += [[y[z1], y[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels1), np.array(labels)

             

In [7]:
# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(10)]
tr_pairs, tr_pair_labels, tr_labels = create_pairs(X_train, y_train, digit_indices )

digit_indices = [np.where(y_test == i)[0] for i in range(10)]
te_pairs, te_pair_labels, te_labels = create_pairs(X_test, y_test, digit_indices)

tr1_pairs, tr1_y = create_addi_pairs(X_train, X_test1)

print(tr_pairs.shape)
print(tr_pair_labels.shape)
print(te_pairs.shape)
print(tr1_pairs.shape)

(1360, 2, 10)
(1360, 2)
(1120, 2, 10)
(500, 2, 10)


In [8]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state = 0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)
X_test1, y_test1 = shuffle(X_test1, y_test1, random_state=0)
print(X_train.shape)

(1242, 10)


In [9]:
# model
# Siamese Network
def siamese(X_input, output_dim, reuse= False):
    with tf.variable_scope('siamese') as scope:
        if (reuse):
            tf.get_variable_scope().reuse_variables()
        #first fully connected layer
        W_fc1 = tf.get_variable('s_wfc1', [10, 16], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('s_bfc1', [16], initializer=tf.constant_initializer(0))
        h_fc1 = tf.nn.relu(tf.matmul(X_input, W_fc1) + b_fc1)
        
        #second fully connected layer
        W_fc2 = tf.get_variable('s_wfc2', [16, 32], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('s_bfc2', [32], initializer=tf.constant_initializer(0))
        h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)
        
        #third fully connected layer
        W_fc3 = tf.get_variable('s_wfc3', [32, output_dim], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc3 = tf.get_variable('s_bfc3', [output_dim], initializer=tf.constant_initializer(0))
        h_fc3 = tf.nn.relu(tf.matmul(h_fc2, W_fc3) + b_fc3)
        
    return h_fc3

In [10]:
# model
# Classifier
def classifier(X_input, input_dim, num_classes, reuse= False):
    with tf.variable_scope('classifier') as scope:
        if (reuse):
            tf.get_variable_scope().reuse_variables()
        #first fully connected layer
        W_fc1 = tf.get_variable('c_wfc1', [input_dim, 32], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('c_bfc1', [32], initializer=tf.constant_initializer(0))
        h_fc1 = tf.nn.relu(tf.matmul(X_input, W_fc1) + b_fc1)
        
        #second fully connected layer
        W_fc2 = tf.get_variable('c_wfc2', [32, 16], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('c_bfc2', [16], initializer=tf.constant_initializer(0))
        h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)
        
        #third fully connected layer
        W_fc3 = tf.get_variable('c_wfc3', [16, num_classes], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc3 = tf.get_variable('c_bfc3', [num_classes], initializer=tf.constant_initializer(0))
        h_fc3 = tf.nn.softmax(tf.matmul(h_fc2, W_fc3) + b_fc3)
        
    return h_fc3

In [11]:
batch_size = 32
num_classes = 14
output_dim = 32
sess = tf.Session()

# placeholder for inputs
X_left = tf.placeholder('float', shape= [None, 10])
X_right = tf.placeholder('float', shape= [None, 10])

# placeholder for labels
Y_left = tf.placeholder('float', shape= [None, 14])
Y_right = tf.placeholder('float', shape= [None, 14])

Y_isSame = tf.placeholder('float', shape= [None, 1])


In [12]:
# model outputs

processed_left = siamese(X_left, output_dim)
processed_right = siamese(X_right, output_dim, reuse=True)

classify_left = classifier(processed_left, output_dim, num_classes, reuse=False)
classify_right = classifier(processed_right,output_dim, num_classes, reuse=True)

In [13]:
print(processed_left.shape)
print(classify_left.shape)

(?, 32)
(?, 14)


In [30]:
# lossses

# crossentropy loss
y_clipped_left = tf.clip_by_value(classify_left, 1e-10, 0.9999999)
y_clipped_right = tf.clip_by_value(classify_right, 1e-10, 0.9999999)
cross_entropy_left = -tf.reduce_mean(tf.reduce_sum(Y_left * tf.log(y_clipped_left)
                         + (1 - Y_left) * tf.log(1 - y_clipped_left), axis=1))
cross_entropy_right = -tf.reduce_mean(tf.reduce_sum(Y_right * tf.log(y_clipped_right)
                         + (1 - Y_right) * tf.log(1 - y_clipped_right), axis=1))
#cross_entropy = (cross_entropy_left + cross_entropy_right)/2.0
cross_entropy = tf.losses.softmax_cross_entropy(Y_left, classify_left)+tf.losses.softmax_cross_entropy(Y_right, classify_right)

print(cross_entropy.shape)

# contrastive loss
y_pred1 = tf.sqrt(tf.reduce_sum(tf.square(processed_left - processed_right), axis=1, keep_dims=True))
y_true1 = Y_isSame
margin = 1
contrastive_loss = tf.reduce_mean(y_true1 * tf.square(y_pred1) + 
                                  (1 - y_true1) * tf.square(tf.maximum(margin - y_pred1, 0)))
print(contrastive_loss.shape)
print(y_pred1.shape)
print(y_true1.shape)

# logcoral loss
n = 32.0
mul1 = tf.matmul(tf.transpose(processed_left),processed_left)
one = processed_left*0+1
mul2 = tf.matmul(tf.transpose(one), processed_left)
sub = tf.matmul(tf.transpose(mul2), mul2)
source = (mul1 - (sub)/n)/(n-1)
source = tf.abs(source)
source = tf.clip_by_value(source, 1e-10,10000)
source1 = tf.log(source)

mul11 = tf.matmul(tf.transpose(processed_right),processed_right)
mul21 = tf.matmul(tf.transpose(one), processed_right)
sub1 = tf.matmul(tf.transpose(mul2), mul2)
target = (mul11 - (sub1)/n)/(n-1)
target = tf.abs(target)
target = tf.clip_by_value(target, 1e-10,10000)
target1 = tf.log(target)
logcoral_loss = (tf.reduce_sum(tf.matmul((source1-target1),(source1-target1)))/(2*32.0))
print(logcoral_loss.shape)

()
()
(?, 1)
(?, 1)
()


In [31]:
tvars = tf.trainable_variables()
s_vars = [var for var in tvars if 's_' in var.name]
c_vars = [var for var in tvars if 'c_' in var.name]
print(len(s_vars))
print(len(c_vars))

6
6


In [32]:
print(tf.get_variable_scope().reuse)
adam = tf.train.AdamOptimizer()
trainer1 = adam.minimize(cross_entropy, var_list=c_vars)
trainer2 = adam.minimize(contrastive_loss, var_list=s_vars)
trainer3 = adam.minimize(logcoral_loss, var_list=s_vars)

False


In [17]:
correct_prediction_left = tf.equal(tf.argmax(Y_left, 1), tf.argmax(classify_left, 1))
correct_prediction_right = tf.equal(tf.argmax(Y_right, 1), tf.argmax(classify_right, 1))

accuracy_left = tf.reduce_mean(tf.cast(correct_prediction_left, tf.float32))
accuracy_right = tf.reduce_mean(tf.cast(correct_prediction_right, tf.float32))

In [18]:
from keras.utils import np_utils
tr_label1 = np_utils.to_categorical(tr_pair_labels[:,0], num_classes=14)
tr_label2 = np_utils.to_categorical(tr_pair_labels[:,1], num_classes=14)
te_label1 = np_utils.to_categorical(te_pair_labels[:,0], num_classes=14)
te_label2 = np_utils.to_categorical(te_pair_labels[:,1], num_classes=14)
print(tr_label1.shape)
print(te_label1.shape)

(1360, 14)
(1120, 14)


In [19]:
y_train_onehot = np_utils.to_categorical(y_train, num_classes=14)
y_test_onehot = np_utils.to_categorical(y_test, num_classes=14)
y_test1_onehot = np_utils.to_categorical(y_test1, num_classes=14)

print(y_train_onehot.shape)

(1242, 14)


In [20]:
print(tr_pair_labels[:,1].max())
print(y_train.max())
print(tr_label1[0:0+32].shape)
print(y_train_onehot[0:100].shape)

9
13
(32, 14)
(100, 14)


In [21]:
print(y_train_onehot)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [33]:
# Start Training
# Start a new TF session
sess = tf.Session()
# Run the initializer
sess.run(tf.global_variables_initializer())

num_batch_same = int(1360/32)
num_batch_class = int(1242/32)
# Training
for i in range(0,2000):
    k = 0 
    avg_cost = 0
    for j in (0,num_batch_same):
        batch_left = tr_pairs[k:k+32,0]
        batch_right = tr_pairs[k:k+32,1]
        label =  tr_labels[k:k+32]
        label = label.reshape(-1, 1)
        k+=32
    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([trainer2, contrastive_loss], feed_dict={X_left: batch_left, X_right: batch_right, Y_isSame: label})
        avg_cost += l / num_batch_same
    print("Epoch:", (i + 1), "contrastive_loss =", "{:.8f}".format(avg_cost))
    
    #avg_cost = 0
    #k=0
    #_, l = sess.run([trainer3, logcoral_loss], feed_dict={X_left: tr1_pairs[:,0], X_right: tr1_pairs[:,1]})
    #print("Epoch:", (i + 1), "logcoral_loss =", "{:.8f}".format(l))
    
    avg_cost = 0
    k=0
    for j in (0,num_batch_same):
        batch_left = X_train[k:k+32]
        batch_right = X_train[k:k+32]
        label_left =  y_train_onehot[k:k+32]
        label_right = y_train_onehot[k:k+32]
        k+=32
    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([trainer1, cross_entropy], feed_dict={X_left: batch_left, X_right: batch_right, Y_left: label_left, Y_right: label_right})
        avg_cost += l / num_batch_same
    print("Epoch:", (i + 1), "cross_entropy =", "{:.8f}".format(avg_cost))
    left_te_acc, correct = sess.run([accuracy_left,classify_left], feed_dict={X_left: X_test, Y_left: y_test_onehot})
    left_te1_acc = sess.run(accuracy_left, feed_dict={X_left: X_test1, Y_left: y_test1_onehot})
    left_tr_acc = sess.run(accuracy_left, feed_dict={X_left: X_train, Y_left: y_train_onehot})
    right_tr_acc = sess.run(accuracy_right, feed_dict={X_right: X_test, Y_right: y_test_onehot})
    right_te_acc = sess.run(accuracy_right, feed_dict={X_right: X_test1, Y_right: y_test1_onehot})
    #print(correct)
    print("Epoch:", (i + 1), "train_accuracy_left =", "{:.8f}".format(left_tr_acc), "Epoch:", (i + 1), "test_accuracy_left =", "{:.8f}".format(left_te_acc))
    print("Epoch:", (i + 1), "domain_accuracy_left =", "{:.8f}".format(left_te1_acc))
    #print("Epoch:", (i + 1), "train_accuracy_right =", "{:.8f}".format(right_tr_acc), "Epoch:", (i + 1), "test_accuracy_right =", "{:.8f}".format(right_te_acc))
    
    print("")

Epoch: 1 contrastive_loss = 0.02380603
Epoch: 1 cross_entropy = 0.25133883
Epoch: 1 train_accuracy_left = 0.05475040 Epoch: 1 test_accuracy_left = 0.06789137
Epoch: 1 domain_accuracy_left = 0.04306220

Epoch: 2 contrastive_loss = 0.02380232
Epoch: 2 cross_entropy = 0.25133559
Epoch: 2 train_accuracy_left = 0.05475040 Epoch: 2 test_accuracy_left = 0.06789137
Epoch: 2 domain_accuracy_left = 0.04306220

Epoch: 3 contrastive_loss = 0.02379726
Epoch: 3 cross_entropy = 0.25133217
Epoch: 3 train_accuracy_left = 0.05475040 Epoch: 3 test_accuracy_left = 0.06789137
Epoch: 3 domain_accuracy_left = 0.04306220

Epoch: 4 contrastive_loss = 0.02378964
Epoch: 4 cross_entropy = 0.25132846
Epoch: 4 train_accuracy_left = 0.05475040 Epoch: 4 test_accuracy_left = 0.06789137
Epoch: 4 domain_accuracy_left = 0.04306220

Epoch: 5 contrastive_loss = 0.02377808
Epoch: 5 cross_entropy = 0.25132444
Epoch: 5 train_accuracy_left = 0.05475040 Epoch: 5 test_accuracy_left = 0.06789137
Epoch: 5 domain_accuracy_left = 0.

Epoch: 55 train_accuracy_left = 0.05475040 Epoch: 55 test_accuracy_left = 0.06789137
Epoch: 55 domain_accuracy_left = 0.04306220

Epoch: 56 contrastive_loss = 0.00157896
Epoch: 56 cross_entropy = 0.24774976
Epoch: 56 train_accuracy_left = 0.05475040 Epoch: 56 test_accuracy_left = 0.06789137
Epoch: 56 domain_accuracy_left = 0.04306220

Epoch: 57 contrastive_loss = 0.00155019
Epoch: 57 cross_entropy = 0.24765319
Epoch: 57 train_accuracy_left = 0.05475040 Epoch: 57 test_accuracy_left = 0.06789137
Epoch: 57 domain_accuracy_left = 0.04306220

Epoch: 58 contrastive_loss = 0.00152345
Epoch: 58 cross_entropy = 0.24755393
Epoch: 58 train_accuracy_left = 0.05475040 Epoch: 58 test_accuracy_left = 0.06789137
Epoch: 58 domain_accuracy_left = 0.04306220

Epoch: 59 contrastive_loss = 0.00149795
Epoch: 59 cross_entropy = 0.24745317
Epoch: 59 train_accuracy_left = 0.05475040 Epoch: 59 test_accuracy_left = 0.06789137
Epoch: 59 domain_accuracy_left = 0.04306220

Epoch: 60 contrastive_loss = 0.00147043
Ep

Epoch: 97 cross_entropy = 0.24597210
Epoch: 97 train_accuracy_left = 0.14814815 Epoch: 97 test_accuracy_left = 0.13897763
Epoch: 97 domain_accuracy_left = 0.09888357

Epoch: 98 contrastive_loss = 0.00101574
Epoch: 98 cross_entropy = 0.24595488
Epoch: 98 train_accuracy_left = 0.14814815 Epoch: 98 test_accuracy_left = 0.13897763
Epoch: 98 domain_accuracy_left = 0.09888357

Epoch: 99 contrastive_loss = 0.00101166
Epoch: 99 cross_entropy = 0.24593732
Epoch: 99 train_accuracy_left = 0.14814815 Epoch: 99 test_accuracy_left = 0.13897763
Epoch: 99 domain_accuracy_left = 0.09888357

Epoch: 100 contrastive_loss = 0.00100798
Epoch: 100 cross_entropy = 0.24591854
Epoch: 100 train_accuracy_left = 0.14814815 Epoch: 100 test_accuracy_left = 0.13897763
Epoch: 100 domain_accuracy_left = 0.09888357

Epoch: 101 contrastive_loss = 0.00100434
Epoch: 101 cross_entropy = 0.24589896
Epoch: 101 train_accuracy_left = 0.14814815 Epoch: 101 test_accuracy_left = 0.13897763
Epoch: 101 domain_accuracy_left = 0.09888

Epoch: 144 train_accuracy_left = 0.14814815 Epoch: 144 test_accuracy_left = 0.13897763
Epoch: 144 domain_accuracy_left = 0.09888357

Epoch: 145 contrastive_loss = 0.00092213
Epoch: 145 cross_entropy = 0.24505730
Epoch: 145 train_accuracy_left = 0.14814815 Epoch: 145 test_accuracy_left = 0.13897763
Epoch: 145 domain_accuracy_left = 0.09888357

Epoch: 146 contrastive_loss = 0.00092099
Epoch: 146 cross_entropy = 0.24503698
Epoch: 146 train_accuracy_left = 0.14814815 Epoch: 146 test_accuracy_left = 0.13897763
Epoch: 146 domain_accuracy_left = 0.09888357

Epoch: 147 contrastive_loss = 0.00091986
Epoch: 147 cross_entropy = 0.24501656
Epoch: 147 train_accuracy_left = 0.14814815 Epoch: 147 test_accuracy_left = 0.13897763
Epoch: 147 domain_accuracy_left = 0.09888357

Epoch: 148 contrastive_loss = 0.00091873
Epoch: 148 cross_entropy = 0.24499609
Epoch: 148 train_accuracy_left = 0.14814815 Epoch: 148 test_accuracy_left = 0.13897763
Epoch: 148 domain_accuracy_left = 0.09888357

Epoch: 149 contrast

Epoch: 198 train_accuracy_left = 0.20048310 Epoch: 198 test_accuracy_left = 0.18370607
Epoch: 198 domain_accuracy_left = 0.18341307

Epoch: 199 contrastive_loss = 0.00084751
Epoch: 199 cross_entropy = 0.24343127
Epoch: 199 train_accuracy_left = 0.20048310 Epoch: 199 test_accuracy_left = 0.18370607
Epoch: 199 domain_accuracy_left = 0.18341307

Epoch: 200 contrastive_loss = 0.00084603
Epoch: 200 cross_entropy = 0.24337486
Epoch: 200 train_accuracy_left = 0.20128824 Epoch: 200 test_accuracy_left = 0.18370607
Epoch: 200 domain_accuracy_left = 0.18341307

Epoch: 201 contrastive_loss = 0.00084469
Epoch: 201 cross_entropy = 0.24331332
Epoch: 201 train_accuracy_left = 0.20209339 Epoch: 201 test_accuracy_left = 0.18370607
Epoch: 201 domain_accuracy_left = 0.18341307

Epoch: 202 contrastive_loss = 0.00084337
Epoch: 202 cross_entropy = 0.24324901
Epoch: 202 train_accuracy_left = 0.20209339 Epoch: 202 test_accuracy_left = 0.18370607
Epoch: 202 domain_accuracy_left = 0.18341307

Epoch: 203 contrast

...

Epoch: 1807 train_accuracy_left = 0.28502417 Epoch: 1807 test_accuracy_left = 0.27076676
Epoch: 1807 domain_accuracy_left = 0.26315790

Epoch: 1808 contrastive_loss = 0.00000000
Epoch: 1808 cross_entropy = 0.22823032
Epoch: 1808 train_accuracy_left = 0.28502417 Epoch: 1808 test_accuracy_left = 0.27076676
Epoch: 1808 domain_accuracy_left = 0.26315790

Epoch: 1809 contrastive_loss = 0.00000000
Epoch: 1809 cross_entropy = 0.22823019
Epoch: 1809 train_accuracy_left = 0.28502417 Epoch: 1809 test_accuracy_left = 0.27076676
Epoch: 1809 domain_accuracy_left = 0.26315790

Epoch: 1810 contrastive_loss = 0.00000000
Epoch: 1810 cross_entropy = 0.22823008
Epoch: 1810 train_accuracy_left = 0.28502417 Epoch: 1810 test_accuracy_left = 0.27076676
Epoch: 1810 domain_accuracy_left = 0.26315790

Epoch: 1811 contrastive_loss = 0.00000000
Epoch: 1811 cross_entropy = 0.22822995
Epoch: 1811 train_accuracy_left = 0.28502417 Epoch: 1811 test_accuracy_left = 0.27076676
Epoch: 1811 domain_accuracy_left = 0.263157

Epoch: 1850 train_accuracy_left = 0.28502417 Epoch: 1850 test_accuracy_left = 0.27076676
Epoch: 1850 domain_accuracy_left = 0.26315790

Epoch: 1851 contrastive_loss = 0.00000000
Epoch: 1851 cross_entropy = 0.22822515
Epoch: 1851 train_accuracy_left = 0.28502417 Epoch: 1851 test_accuracy_left = 0.27076676
Epoch: 1851 domain_accuracy_left = 0.26315790

Epoch: 1852 contrastive_loss = 0.00000000
Epoch: 1852 cross_entropy = 0.22822503
Epoch: 1852 train_accuracy_left = 0.28502417 Epoch: 1852 test_accuracy_left = 0.27076676
Epoch: 1852 domain_accuracy_left = 0.26315790

Epoch: 1853 contrastive_loss = 0.00000000
Epoch: 1853 cross_entropy = 0.22822494
Epoch: 1853 train_accuracy_left = 0.28502417 Epoch: 1853 test_accuracy_left = 0.27076676
Epoch: 1853 domain_accuracy_left = 0.26315790

Epoch: 1854 contrastive_loss = 0.00000000
Epoch: 1854 cross_entropy = 0.22822481
Epoch: 1854 train_accuracy_left = 0.28502417 Epoch: 1854 test_accuracy_left = 0.27076676
Epoch: 1854 domain_accuracy_left = 0.263157

Epoch: 1892 cross_entropy = 0.22822037
Epoch: 1892 train_accuracy_left = 0.28502417 Epoch: 1892 test_accuracy_left = 0.27076676
Epoch: 1892 domain_accuracy_left = 0.26315790

Epoch: 1893 contrastive_loss = 0.00000000
Epoch: 1893 cross_entropy = 0.22822025
Epoch: 1893 train_accuracy_left = 0.28502417 Epoch: 1893 test_accuracy_left = 0.27076676
Epoch: 1893 domain_accuracy_left = 0.26315790

Epoch: 1894 contrastive_loss = 0.00000000
Epoch: 1894 cross_entropy = 0.22822014
Epoch: 1894 train_accuracy_left = 0.28502417 Epoch: 1894 test_accuracy_left = 0.27076676
Epoch: 1894 domain_accuracy_left = 0.26315790

Epoch: 1895 contrastive_loss = 0.00000000
Epoch: 1895 cross_entropy = 0.22822002
Epoch: 1895 train_accuracy_left = 0.28502417 Epoch: 1895 test_accuracy_left = 0.27076676
Epoch: 1895 domain_accuracy_left = 0.26315790

Epoch: 1896 contrastive_loss = 0.00000000
Epoch: 1896 cross_entropy = 0.22821992
Epoch: 1896 train_accuracy_left = 0.28502417 Epoch: 1896 test_accuracy_left = 0.27076676
Epoc

Epoch: 1943 contrastive_loss = 0.00000000
Epoch: 1943 cross_entropy = 0.22821456
Epoch: 1943 train_accuracy_left = 0.28502417 Epoch: 1943 test_accuracy_left = 0.27076676
Epoch: 1943 domain_accuracy_left = 0.26315790

Epoch: 1944 contrastive_loss = 0.00000000
Epoch: 1944 cross_entropy = 0.22821446
Epoch: 1944 train_accuracy_left = 0.28502417 Epoch: 1944 test_accuracy_left = 0.27076676
Epoch: 1944 domain_accuracy_left = 0.26315790

Epoch: 1945 contrastive_loss = 0.00000000
Epoch: 1945 cross_entropy = 0.22821435
Epoch: 1945 train_accuracy_left = 0.28502417 Epoch: 1945 test_accuracy_left = 0.27076676
Epoch: 1945 domain_accuracy_left = 0.26315790

Epoch: 1946 contrastive_loss = 0.00000000
Epoch: 1946 cross_entropy = 0.22821424
Epoch: 1946 train_accuracy_left = 0.28502417 Epoch: 1946 test_accuracy_left = 0.27076676
Epoch: 1946 domain_accuracy_left = 0.26315790

Epoch: 1947 contrastive_loss = 0.00000000
Epoch: 1947 cross_entropy = 0.22821413
Epoch: 1947 train_accuracy_left = 0.28502417 Epoch:

Epoch: 1999 train_accuracy_left = 0.28502417 Epoch: 1999 test_accuracy_left = 0.27076676
Epoch: 1999 domain_accuracy_left = 0.26315790

Epoch: 2000 contrastive_loss = 0.00000000
Epoch: 2000 cross_entropy = 0.22820828
Epoch: 2000 train_accuracy_left = 0.28502417 Epoch: 2000 test_accuracy_left = 0.27076676
Epoch: 2000 domain_accuracy_left = 0.26315790



In [28]:
n = 122
print(y_train_onehot[n])
correct = sess.run([accuracy_left, classify_left, cross_entropy_left], feed_dict={X_left: X_train[n:n+1], Y_left: y_train_onehot[n:n+1]})
print(correct)

correct1 = sess.run([accuracy_right, classify_right, cross_entropy_right], feed_dict={X_right: X_train[n:n+1], Y_right: y_train_onehot[n:n+1]})
print(correct1)

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.0, array([[9.4366413e-01, 5.9523277e-28, 4.5179157e-31, 1.3229128e-32,
        1.6827399e-28, 1.3650492e-14, 3.0780107e-33, 6.4932131e-28,
        5.6335837e-02, 4.4336421e-31, 7.2253318e-33, 6.1677170e-26,
        8.5496753e-34, 6.5270523e-31]], dtype=float32), 25.960258]
[0.0, array([[9.4366413e-01, 5.9523277e-28, 4.5179157e-31, 1.3229128e-32,
        1.6827399e-28, 1.3650492e-14, 3.0780107e-33, 6.4932131e-28,
        5.6335837e-02, 4.4336421e-31, 7.2253318e-33, 6.1677170e-26,
        8.5496753e-34, 6.5270523e-31]], dtype=float32), 25.960258]
